In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
data = Path("../data")
plot_path = data / "plots"
map_path = data / "graph3_map"

# Load users dataset

302,673 unique users, 55,280 unique location strings

In [2]:
users = pd.read_csv(data / "UserList.csv")
print("Number of unique users: {:,}".format(users["user_id"].nunique()))
print("Number of unique locations: {:,}".format(users["location"].nunique()))
users

Number of unique users: 302,673
Number of unique locations: 55,280


,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes
0,karthiga,2255153,3,49,1,0,0,55.31,Female,"Chennai, India",1990-04-29,NaN,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391.0
1,RedvelvetDaisuki,1897606,61,396,39,0,206,118.07,Female,Manila,1995-01-01,NaN,2012-12-13,1900-05-13 02:47:00,6.78,80.0,7094.0
2,Damonashu,37326,45,195,27,25,59,83.70,Male,"Detroit,Michigan",1991-08-01,NaN,2008-02-13,1900-03-24 12:48:00,6.15,6.0,4936.0
3,bskai,228342,25,414,2,5,11,167.16,Male,"Nayarit, Mexico",1990-12-14,NaN,2009-08-31,2014-05-12 16:35:00,8.27,1.0,10081.0
4,shuzzable,2347781,36,72,16,2,25,35.48,NaN,NaN,NaN,NaN,2013-03-25,2015-09-09 21:54:00,9.06,7.0,2154.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302670,ScruffyPuffy,3119025,0,27,0,0,0,7.92,NaN,NaN,NaN,NaN,2013-09-06,2014-10-10 09:04:00,0.00,0.0,477.0
302671,Torasori,3975907,22,239,0,4,176,86.88,Male,"Latvia, Riga",1998-11-18,NaN,2014-07-30,2018-05-24 21:34:46,8.98,47.0,5313.0
302672,onpc,1268417,5,169,2,5,24,38.36,Male,NaN,NaN,NaN,2012-04-23,2016-12-28 14:35:00,7.72,0.0,2280.0
302673,HMicca,1289601,11,73,2,2,16,119.97,Female,"Birmingham, England",1995-08-12,NaN,2012-05-05,2012-11-15 08:10:00,8.89,11.0,7049.0


In [3]:
# Keep users with non-null location
users = users[users["location"].notnull()]

# strip all locations
users.loc[:, "location"] = users.loc[:, "location"].str.strip()

print("Number of unique users: {:,}".format(users["user_id"].nunique()))
print("Number of unique locations: {:,}".format(users["location"].nunique()))

Number of unique users: 156,774
Number of unique locations: 53,762


/tmp/ipykernel_11138/1065557805.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users.loc[:, "location"] = users.loc[:, "location"].str.strip()


# Map location strings to country names

In [4]:
from functools import partial
from geopy.geocoders import Nominatim
from deep_translator import GoogleTranslator

import country_converter as coco

coco.logging.getLogger().setLevel(coco.logging.ERROR)

translator = GoogleTranslator(source="auto", target="en")
geolocator = Nominatim(user_agent="GoogleV3")
geocode = partial(geolocator.geocode, language="en")


def findCountry(location):
    if location.isdigit():
        print(location, "(is digit)")
        return ""

    try:
        result = geocode(location, timeout=5)
    except:
        print(location, "(geocode)")
        return ""
    if result is not None:
        country = result.address.split(",")[-1].strip()
        return country

    country = coco.convert(names=location, to="name_short")
    if country != "not found":
        return country

    translated = translator.translate(location)
    if translated is None:
        print(location, "(translation not found)")
        return ""
    if translated == location:
        print(location, "(not found)")
        return ""

    try:
        result = geocode(translated, timeout=5)
    except:
        print(location, "->", translated, "(geocode)")
        return ""
    if result is not None:
        country = result.address.split(",")[-1].strip()
        return country

    country = coco.convert(names=translated, to="name_short")
    if country != "not found":
        return country
    print(location, "->", translated, "(not found)")
    return ""

# Cleaning countries

- Fix the incorrect mappings
- Add a "ambiguous" class for locations that are not precise enough and that may correspond to cities of the same name but in different countries
- For lists of countries, ...

## Merge with users dataset

In [5]:
df_ltc_clean = pd.read_csv(map_path / "location_to_country_clean.csv")
# Transform df_ltc_clean to a dictionary with key "location" and value "country"
dict_ltc_clean = df_ltc_clean.set_index("location").to_dict()["country"]
dict_ltc_clean

{'Afghan': 'Afghanistan',
 'Albania': 'Albania',
 'Albania / Tirana': 'Albania',
 'Albania, Tirana': 'Albania',
 'Albania,Tirana': 'Albania',
 'Durres': 'Albania',
 'Durres, Albania': 'Albania',
 'Kavaje, Albania': 'Albania',
 'Tirana , Albania': 'Albania',
 'Tirana, Albania': 'Albania',
 'Vlore, Albania': 'Albania',
 'tirana,Albania': 'Albania',
 'tirane': 'Albania',
 'ALG': 'Algeria',
 'ALGERIA/BMR': 'Algeria',
 'ANNABA': 'Algeria',
 'Alger,Algeria': 'Algeria',
 'Algeria': 'Algeria',
 'Algeria ,Chlef': 'Algeria',
 'Algeria, Alger': 'Algeria',
 'Algeria, Algiers': 'Algeria',
 'Algeria, Chlef': 'Algeria',
 'Algeria,Barika': 'Algeria',
 'Algeria,Setif': 'Algeria',
 'Algeria-mascara': 'Algeria',
 'Algeria/Tebessa/Cheria': 'Algeria',
 'Algeria_Batna_Bou Zoran': 'Algeria',
 'Algiers': 'Algeria',
 'Algiers, Algeria': 'Algeria',
 'Algérie,Jijel': 'Algeria',
 'Annaba': 'Algeria',
 'Annaba, Algeria': 'Algeria',
 'Annaba, algeria': 'Algeria',
 'Batna': 'Algeria',
 'Batna,Algeria': 'Algeria',
 '

In [6]:
users["country"] = users["location"].map(dict_ltc_clean)
users

/tmp/ipykernel_11138/2562388302.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users["country"] = users["location"].map(dict_ltc_clean)


,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,country
0,karthiga,2255153,3,49,1,0,0,55.31,Female,"Chennai, India",1990-04-29,NaN,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391.0,India
1,RedvelvetDaisuki,1897606,61,396,39,0,206,118.07,Female,Manila,1995-01-01,NaN,2012-12-13,1900-05-13 02:47:00,6.78,80.0,7094.0,Philippines
2,Damonashu,37326,45,195,27,25,59,83.70,Male,"Detroit,Michigan",1991-08-01,NaN,2008-02-13,1900-03-24 12:48:00,6.15,6.0,4936.0,United States
3,bskai,228342,25,414,2,5,11,167.16,Male,"Nayarit, Mexico",1990-12-14,NaN,2009-08-31,2014-05-12 16:35:00,8.27,1.0,10081.0,Mexico
5,terune_uzumaki,327311,5,5,0,0,0,15.20,Female,"Malaysia, Kuantan",1998-08-24,NaN,2010-05-10,2012-10-18 19:06:00,9.70,6.0,920.0,Malaysia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302658,Queenjdlols,3495411,4,123,6,4,49,22.55,Female,Head in the clouds body on the ground,NaN,NaN,2014-01-16,1900-04-29 00:19:00,6.77,3.0,1375.0,NaN
302662,TheClockworkGuy,4273353,5,77,8,12,61,48.03,Male,Yharnam,NaN,NaN,2014-11-23,1900-05-22 02:49:00,6.85,0.0,2826.0,United States
302668,ichinitan,4531649,3,24,5,0,246,8.72,NaN,"Alexandria, Virginia",2000-05-12,NaN,2015-04-01,2017-10-12 14:22:00,9.46,0.0,528.0,United States
302671,Torasori,3975907,22,239,0,4,176,86.88,Male,"Latvia, Riga",1998-11-18,NaN,2014-07-30,2018-05-24 21:34:46,8.98,47.0,5313.0,Latvia


# Cleaning users

## Remove users with undefined country

In [7]:
num_unique_users = users["user_id"].nunique()

# Users whose country is undefined (country is "" or is null)
num_undefined = users[users["country"].isnull() | (users["country"] == "")]["user_id"].nunique()
print("Number of users whose country is undefined: {:,} ({:.2f}%)".format(num_undefined, 100 * num_undefined / num_unique_users))


users = users[users["country"].notnull() & (users["country"] != "")]
print("Number of users whose country is defined: {:,}".format(users["user_id"].nunique()))

Number of users whose country is undefined: 20,499 (13.08%)
Number of users whose country is defined: 136,275


## Duplicate the rows of the users who are located in several countries

In [8]:
# for all the users who have a "country" column of the form "['', '']", remove the [] and ''. For instance, "['Finland', 'Sweden']" becomes "Finland, Sweden"
num_unique_users = users["user_id"].nunique()

users.loc[:, "country"] = (
    users.loc[:, "country"]
    .str.replace(r"^\[\'", "")
    .str.replace(r"\'\]$", "")
    .str.replace(r"\'\,\s\'", ", ")
)

# for the users who have a "country" column of the form "country1, country2, ..., countryn", duplicate the user n-1 times and assign the country1, country2, ..., countryn to the new users
users_multiple_countries = users[users["country"].str.contains(",")]
users_multiple_countries.loc[:, "country"] = users_multiple_countries.loc[:, "country"].str.split(", ")
users_multiple_countries = users_multiple_countries.explode("country")

# merge the results with the original dataframe
users = pd.concat(
    [users[~users["country"].str.contains(",")], users_multiple_countries], axis=0
)

# Check that the number of unique users is the same after the operation
assert users["user_id"].nunique() == num_unique_users
users

/tmp/ipykernel_11138/2748047192.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r"^\[\'", "")
/tmp/ipykernel_11138/2748047192.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r"\'\]$", "")
/tmp/ipykernel_11138/2748047192.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r"\'\,\s\'", ", ")
/tmp/ipykernel_11138/2748047192.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users.loc[:, "country"] = (
/tmp/ipykernel_11138/2748047192.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,country
0,karthiga,2255153,3,49,1,0,0,55.31,Female,"Chennai, India",1990-04-29,NaN,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391.0,India
1,RedvelvetDaisuki,1897606,61,396,39,0,206,118.07,Female,Manila,1995-01-01,NaN,2012-12-13,1900-05-13 02:47:00,6.78,80.0,7094.0,Philippines
2,Damonashu,37326,45,195,27,25,59,83.70,Male,"Detroit,Michigan",1991-08-01,NaN,2008-02-13,1900-03-24 12:48:00,6.15,6.0,4936.0,United States
3,bskai,228342,25,414,2,5,11,167.16,Male,"Nayarit, Mexico",1990-12-14,NaN,2009-08-31,2014-05-12 16:35:00,8.27,1.0,10081.0,Mexico
5,terune_uzumaki,327311,5,5,0,0,0,15.20,Female,"Malaysia, Kuantan",1998-08-24,NaN,2010-05-10,2012-10-18 19:06:00,9.70,6.0,920.0,Malaysia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286854,AdamNyandere,4539831,11,43,7,1,25,47.37,Male,Singapore / Malaysia,1984-11-18,NaN,2015-04-05,2018-05-23 07:28:00,9.07,60.0,3967.0,Singapore
289896,Roxas_the_key,242758,4,63,19,0,2,42.19,Female,"Germany,Greece",1991-09-11,NaN,2009-10-05,2012-09-27 12:01:00,8.63,0.0,2521.0,Germany
289896,Roxas_the_key,242758,4,63,19,0,2,42.19,Female,"Germany,Greece",1991-09-11,NaN,2009-10-05,2012-09-27 12:01:00,8.63,0.0,2521.0,Greece
297997,Tachibana-chan,5078714,17,105,2,4,37,24.65,Female,India/Saudi Arabia,2004-01-25,NaN,2015-12-27,1900-05-02 03:48:00,7.32,0.0,1530.0,India


# Countries centroids dataset ([country_centroids.csv](country_centroids.csv))

- Fix country names of the mappings to match the "COUNTRY" columns
- Add missing country-centroids pairs: Taiwan, Kosovo, Null Island

In [9]:
centroids = pd.read_csv(map_path / "country_centroids.csv")
centroids

,longitude,latitude,COUNTRY,ISO,COUNTRYAFF,AFF_ISO
0,-170.700732,-14.305712,American Samoa,AS,United States,US
1,166.638003,19.302046,United States Minor Outlying Islands,UM,United States,US
2,-159.787689,-21.222613,Cook Islands,CK,New Zealand,NZ
3,-149.400417,-17.674684,French Polynesia,PF,France,FR
4,-169.868781,-19.052309,Niue,NU,New Zealand,NZ
...,...,...,...,...,...,...
247,-3.651625,40.365008,Spain,ES,Spain,ES
248,-16.537994,28.297665,Canarias,ES,Spain,ES
249,23.700000,121.000000,Taiwan,TW,Taiwan,TW
250,42.600000,20.900000,Kosovo,XK,Kosovo,XK


In [10]:
# Print the countries who have affiliated territories 
countries_with_affiliations = centroids.groupby("COUNTRYAFF").filter(lambda x: len(x) > 1)["COUNTRYAFF"].unique()
countries_with_affiliations

array(['United States', 'New Zealand', 'France', 'United Kingdom',
       'Netherlands', 'Denmark', 'Norway', 'Australia', 'Spain'],
      dtype=object)

Some countries have affiliated territories, such as the United Kingdom, France, Spain, Netherlands, etc. We will consider these territories as part of the country they are affiliated to.

In [11]:
centroids[centroids["COUNTRYAFF"] == "France"]

,longitude,latitude,COUNTRY,ISO,COUNTRYAFF,AFF_ISO
3,-149.400417,-17.674684,French Polynesia,PF,France,FR
9,-178.127356,-14.283442,Wallis and Futuna,WF,France,FR
43,-53.322323,3.857430,French Guiana,GF,France,FR
45,-61.543823,16.244200,Guadeloupe,GP,France,FR
50,-61.014324,14.642697,Martinique,MQ,France,FR
56,-62.830516,17.905617,Saint Barthelemy,BL,France,FR
60,-63.066785,18.078012,Saint Martin,MF,France,FR
61,-56.324654,46.951539,Saint Pierre and Miquelon,PM,France,FR
105,42.743748,-17.064492,Juan De Nova Island,TF,France,FR
116,69.546870,-49.263297,French Southern Territories,TF,France,FR


In [12]:
centroids[centroids["COUNTRYAFF"] == "United States"]

,longitude,latitude,COUNTRY,ISO,COUNTRYAFF,AFF_ISO
0,-170.700732,-14.305712,American Samoa,AS,United States,US
1,166.638003,19.302046,United States Minor Outlying Islands,UM,United States,US
54,-66.494253,18.216224,Puerto Rico,PR,United States,US
67,-64.761553,17.738010,US Virgin Islands,VI,United States,US
154,-96.331617,38.820809,United States,US,United States,US
240,144.780245,13.445430,Guam,GU,United States,US
244,145.741197,15.178064,Northern Mariana Islands,MP,United States,US


In [13]:
# keep only COUNTRY and COUNTRYAFF
centroids_temp = centroids[["COUNTRY", "COUNTRYAFF"]]
centroids_temp

,COUNTRY,COUNTRYAFF
0,American Samoa,United States
1,United States Minor Outlying Islands,United States
2,Cook Islands,New Zealand
3,French Polynesia,France
4,Niue,New Zealand
...,...,...
247,Spain,Spain
248,Canarias,Spain
249,Taiwan,Taiwan
250,Kosovo,Kosovo


In [14]:
# Add a "COUNTRYAFF" column to users
users_country_aff = users.merge(centroids_temp, left_on="country", right_on="COUNTRY", how="inner")

# The rows which were not merged correspond to users with "ambiguous" country
assert len(users) - len(users_country_aff) == len(users[users["country"]=="ambiguous"])

# Cleaning: remove the "COUNTRY" column and rename "COUNTRYAFF" to "country_aff"
users_country_aff = users_country_aff.drop(columns=["COUNTRY"]).rename(columns={"COUNTRYAFF": "country_aff"})
users_country_aff

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,country,country_aff
0,karthiga,2255153,3,49,1,0,0,55.31,Female,"Chennai, India",1990-04-29,NaN,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391.0,India,India
1,K_P,6553594,32,419,15,3,122,181.52,Male,India,1998-02-06,NaN,2017-09-13,2018-05-14 22:22:07,9.05,140.0,11019.0,India,India
2,thomassimpsons,4778210,8,68,2,5,6,47.17,Male,"India,Mumbai",2000-08-16,NaN,2015-07-29,1900-04-14 06:46:00,6.63,0.0,2833.0,India,India
3,BLACK1189,6688908,2,220,2,0,4,67.40,Male,"india, kerala",1993-05-27,NaN,2017-10-29,1900-04-22 10:04:00,6.71,0.0,3998.0,India,India
4,Leon07,5037170,22,237,46,5,221,126.81,Male,Kerala,NaN,NaN,2015-12-07,1900-05-02 03:28:00,7.71,26.0,7698.0,India,India
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136040,prof_salt,4461555,6,240,12,6,349,81.15,Male,"Tarawa, Kiribati",1999-04-10,NaN,2015-02-09,1900-05-12 01:20:00,7.55,0.0,5080.0,Kiribati,Kiribati
136041,marijuanamoneyho,3662727,4,13,0,14,0,16.00,Male,Afghan,1930-01-01,NaN,2014-03-28,2014-07-03 14:28:00,7.03,0.0,957.0,Afghanistan,Afghanistan
136042,flora4u,224257,0,0,0,0,0,0.00,Female,dakar,1985-02-09,NaN,2009-08-22,2013-09-13 13:01:00,0.00,0.0,0.0,Senegal,Senegal
136043,kuranlover7891,303432,11,23,0,3,25,16.59,Female,"Nigeria, Benin",NaN,NaN,2010-03-11,2013-06-30 02:46:00,8.00,0.0,993.0,Benin,Benin


In [15]:
# Users who are located in an affiliated territory
users_country_aff[users_country_aff["country"] != users_country_aff["country_aff"]]

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,country,country_aff
120187,Niichi01,4353873,1,66,10,2,118,21.11,Female,"Camuy, Puerto Rico",1997-09-01,NaN,2014-12-27,1900-05-07 17:00:00,8.49,0.0,1213.0,Puerto Rico,United States
120188,Kito-kun,1971992,4,290,5,0,93,93.64,Male,"Aguadilla, Puerto Rico",1994-01-16,NaN,2012-12-31,1900-03-23 07:55:00,9.50,59.0,5372.0,Puerto Rico,United States
120189,_mayumih,403463,4,29,1,1,2,16.85,Female,Puerto Rico,1994-11-14,NaN,2010-11-02,2011-01-03 21:03:00,9.15,0.0,1004.0,Puerto Rico,United States
120190,Kandomaru,202008,8,1006,7,112,70,216.61,Male,Puerto Rico,1986-05-11,NaN,2009-07-03,2018-05-18 02:13:36,6.99,24.0,12257.0,Puerto Rico,United States
120191,KurosakiLeno,269642,6,239,0,8,15,72.08,Male,Japan Puerto Rico,1930-04-07,NaN,2009-12-14,1900-03-16 08:15:00,8.04,0.0,4385.0,Puerto Rico,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136032,Gennady23,4167817,0,0,0,0,0,0.00,Male,"Turks And Caicos Island, Providenciales",1998-01-13,NaN,2014-10-11,2014-10-11 00:50:00,0.00,0.0,0.0,Turks and Caicos Islands,United Kingdom
136036,CraneTYS,1709381,3,75,2,0,12,18.37,Male,"Bouvet Island, Norway",NaN,NaN,2012-10-31,2013-08-17 15:49:00,8.00,0.0,1039.0,Bouvet Island,Norway
136037,AKAaseria21,5068712,4,3,0,0,2,4.55,Female,Anguilla,2003-01-01,NaN,2015-12-22,2016-07-03 09:59:00,8.88,4.0,273.0,Anguilla,United Kingdom
136038,Turboo_Troll,445496,0,1,0,0,0,0.05,Male,Pń,1989-02-27,NaN,2011-02-06,2011-09-08 12:58:00,0.00,0.0,3.0,Pitcairn,United Kingdom


In [16]:
print("Number of users who have a defined country: {:,}".format(users_country_aff["user_id"].nunique()))
users_country_aff.to_csv(map_path / "users_country_aff.csv", index=False)

Number of users who have a defined country: 135,917
